In [82]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import numpy as np 
import pandas as pd

In [84]:
dataset_path = "./dataset.csv"

df = pd.read_csv(
    dataset_path,
    # dtype is specified to prevent a DtypeWarning
    dtype={
        "full_query": str,
        "label": int,
        "statement_type": str,
        "query_template_id": str,
        "attack_payload": str,
        "attack_id": str,
        "attack_technique": str,
        "attack_desc": str,
        "split": str,
        "sqlmap_status" : str,
        "attack_stage" : str,
        "tamper_method" : str

    },
)

display(df)


,full_query,label,attack_payload,attack_desc,attack_stage,tamper_method,sqlmap_status,statement_type,query_template_id,attack_id,attack_technique,split
0,"SELECT id, ident, name FROM airport WHERE iso_...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S6,NaN,NaN,test
1,"SELECT a.id, a.name, a.municipality, a.iso_cou...",1,NaN,NaN,recon,sleep2getlock,NaN,select,airport-S10,error-2,error,test
2,"SELECT * FROM airport WHERE continent = ""EU') ...",1,NaN,NaN,recon,multiplespaces,NaN,select,airport-S8,error-3,error,test
3,"INSERT INTO runways (airport_ref, airport_iden...",1,NaN,NaN,recon,lowercase,NaN,insert,airport-I9,error-7,error,test
4,"UPDATE airport SET wikipedia_link = ""https://e...",1,NaN,NaN,recon,equaltolike,NaN,update,airport-U3,error-6,error,test
...,...,...,...,...,...,...,...,...,...,...,...,...
14900,"DELETE FROM airport WHERE name = ""Costa Esmera...",0,NaN,NaN,NaN,NaN,NaN,delete,airport-D2,NaN,NaN,train
14901,"SELECT icao_code, iata_code, local_code FROM a...",0,NaN,NaN,NaN,NaN,NaN,select,airport-S3,NaN,NaN,train
14902,"UPDATE airport SET wikipedia_link = ""https://e...",1,NaN,NaN,recon,commentbeforeparentheses,NaN,update,airport-U3,error-6,error,train
14903,"INSERT INTO runways (airport_ref, airport_iden...",1,NaN,NaN,recon,lowercase,NaN,insert,airport-I9,error-7,error,train


In [85]:
def display_attack_normal_per_set(df : pd.DataFrame):
    df_train_atk = df[(df["label"] == 1) & (df["split"] == "train")]  
    df_train_normal = df[(df["label"] == 0) & (df["split"] == "train")]  

    df_test_atk = df[(df["label"] == 1) & (df["split"] == "test")]  
    df_test_normal = df[(df["label"] == 0) & (df["split"] == "test")]  

    print(f"Number of attacks in train: {df_train_atk.shape[0]}")
    print(f"Number of normal in train: {df_train_normal.shape[0]}")
    print(f"Number of attacks in test: {df_test_atk.shape[0]}")
    print(f"Number of normal in test: {df_test_normal.shape[0]}")

display_attack_normal_per_set(df)

Number of attacks in train: 4317
Number of normal in train: 5577
Number of attacks in test: 2525
Number of normal in test: 2486


In [86]:
def display_ids_per_set(df: pd.DataFrame): 
    df_train = df[df["split"] == "train"]
    df_test = df[df["split"] == "test"]

    tids_train  = df_train['query_template_id'].unique()
    tids_test = df_test['query_template_id'].unique()
    tids_complement = set(tids_test) - set(tids_train)
    print(f"Template IDS in df train: {len(tids_train)}, {tids_train}")
    print(f"Template IDS in df test:   {len(tids_test)}, {tids_test}")
    print(f"Template IDs in train but NOT in test: {len(tids_complement)}, {sorted(tids_complement)}")
    
display_ids_per_set(df)

Template IDS in df train: 42, ['airport-D5' 'airport-S12' 'airport-S10' 'airport-S4' 'airport-S15'
 'airport-S11' 'airport-D4' 'airport-I6' 'airport-U4' 'airport-D7'
 'airport-I5' 'airport-D6' 'airport-S5' 'airport-I7' 'airport-I9'
 'airport-S2' 'airport-U3' 'airport-S6' 'airport-S16' 'airport-S8'
 'airport-U8' 'airport-S17' 'airport-U7' 'airport-U2' 'airport-S1'
 'airport-S7' 'airport-S13' 'airport-S3' 'airport-S9' 'airport-U9'
 'airport-I3' 'airport-U6' 'airport-D1' 'airport-I2' 'airport-D2'
 'airport-I8' 'airport-I1' 'airport-I10' 'airport-D3' 'airport-U1'
 'airport-D8' 'airport-U5']
Template IDS in df test:   44, ['airport-S6' 'airport-S10' 'airport-S8' 'airport-I9' 'airport-U3'
 'airport-D1' 'airport-I5' 'airport-I4' 'airport-S15' 'airport-S1'
 'airport-S14' 'airport-S16' 'airport-S17' 'airport-S9' 'airport-D6'
 'airport-D8' 'airport-U7' 'airport-D5' 'airport-I7' 'airport-S11'
 'airport-S13' 'airport-S3' 'airport-U6' 'airport-S12' 'airport-S4'
 'airport-S5' 'airport-S7' 'airport-S